# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages
import re, os, glob, csv, json
import pandas as pd
import numpy as np
import cassandra

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root, '*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline = '') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting = csv.QUOTE_ALL, skipinitialspace = True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline = '') as f:
    writer = csv.writer(f, dialect = 'myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine  (127.0.0.1)

from cassandra.cluster import Cluster
try:
    # This should make a connection to a Cassandra instance your local machine  (127.0.0.1)
    cluster = Cluster(['127.0.0.1'])
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e: print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 

try:
    session.execute('''
                        CREATE KEYSPACE IF NOT EXISTS sparkify 
                        WITH REPLICATION = { 'class' : 'SimpleStrategy', 
                                             'replication_factor' : 1 }
                    ''')
except Exception as e: print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try: session.set_keyspace('sparkify')
except Exception as e: print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1: Version: 1
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338 and itemInSession = 4

## The following pandas dataframe is set as debugger
df = pd.read_csv('event_datafile_new.csv')
df.head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Rokia TraorÃÂ©,Stefany,F,0,White,274.88608,free,"Lubbock, TX",693,Zen,83
1,Camila,Tucker,M,1,Garrison,230.81751,free,"Oxnard-Thousand Oaks-Ventura, CA",555,Abrazame (Version Acustica),40
2,Carl Thomas,Tucker,M,0,Garrison,196.67546,free,"Oxnard-Thousand Oaks-Ventura, CA",698,You Ain't Right (Album Version),40
3,N.E.R.D.,James,M,0,Martin,242.99057,free,"Dallas-Fort Worth-Arlington, TX",78,Provider (Remix Radio Edit),79
4,Lil Jon / The East Side Boyz / DJ Flexx,Jacqueline,F,3,Lynch,285.30893,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Aww Skeet Skeet,29


In [9]:
## Query 1: Version: 3
## Reference: https://stackoverflow.com/questions/37319408/cql-error-invalidrequest-code-2200-invalid-query-message-unconfigured-table
## Reference: https://docs.datastax.com/en/archived/cql/3.3/cql/cql_using/useSimplePrimaryKeyConcept.html#useSimplePrimaryKeyConcept
## Reference: https://docs.datastax.com/en/archived/cql/3.3/cql/cql_reference/cqlCreateTable.html#tabProp
## Reference: https://www.guru99.com/cassandra-table-create-alter-drop-truncate.html
## Reference: https://shermandigital.com/blog/designing-a-cassandra-data-model/

query = "CREATE TABLE IF NOT EXISTS session_item_info"
query = query + '''
                    (sessionid int, iteminsession int, userid int, artist text, song text, length float, 
                     PRIMARY KEY ((sessionid, iteminsession), userid))
                '''
try: session.execute(query)
except Exception as e: print(e)

In [10]:
## Query 1: Version: 3

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item_info(sessionid, iteminsession, userid, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Query 1: Version: 1
## SELECT statement to verify the data was entered into the table

query = '''
            SELECT artist, song, length FROM session_item_info 
            WHERE sessionid = 338 AND iteminsession = 4
        '''
try: rows = session.execute(query)
except Exception as e: print(e)
    
for row in rows: print((row.artist, row.song, row.length))

('Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875)


In [12]:
## Query 1: Version: 2
## SELECT statement to verify the data was entered into the table

query = '''
            SELECT * FROM session_item_info 
            WHERE sessionid = 338 AND iteminsession = 4
        '''
try: rows = session.execute(query)
except Exception as e: print(e)
    
for row in rows: print(row)

Row(sessionid=338, iteminsession=4, userid=50, artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## Query 2: Version: 1
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session_info"
query = query + '''
                    (userid int, sessionid int, iteminsession int, artist text, song text, user text, 
                     PRIMARY KEY ((userid, sessionid), iteminsession))
                '''
try: session.execute(query)
except Exception as e: print(e)

In [14]:
## Query 2: Version: 1

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_info(userid, sessionid, iteminsession, artist, song, user)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        
        name = line[1] + ' ' + line[4]
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], name))

In [15]:
## Query 2: Version: 1

## The following pandas dataframe is set as debugger
df = pd.read_csv('event_datafile_new.csv')
df[(df['userId'] == 10) & (df['sessionId'] == 182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1918,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
1919,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
1920,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
1921,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [16]:
## Query 2: Version: 1
## SELECT statement to verify the data was entered into the table

query = '''
            SELECT artist, song, user FROM user_session_info 
            WHERE userid = 10 AND sessionid = 182
        '''
try: rows = session.execute(query)
except Exception as e: print(e)

for row in rows: print((row.artist, row.song, row.user))

('Down To The Bone', "Keep On Keepin' On", 'Sylvie Cruz')
('Three Drives', 'Greece 2000', 'Sylvie Cruz')
('Sebastien Tellier', 'Kilometer', 'Sylvie Cruz')
('Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie Cruz')


In [17]:
## Query 3: Version: 1
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_listened_by_users"
query = query + '''
                    (song text, user text, 
                     PRIMARY KEY (song, user))
                '''
try: session.execute(query)
except Exception as e: print(e)

In [18]:
## Query 3: Version: 1

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listened_by_users(song, user)"
        query = query + " VALUES (%s, %s)"
        
        name = line[1] + ' ' + line[4]
        session.execute(query, (line[9], name))

In [19]:
## Query 3: Version: 1
## SELECT statement to verify the data was entered into the table

query = '''
            SELECT user FROM song_listened_by_users 
            WHERE song = 'All Hands Against His Own'
        '''
try: rows = session.execute(query)
except Exception as e: print(e)
    
for row in rows: print((row.user))

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the sessions
tables = ['session_item_info', 'user_session_info', 'song_listened_by_users']

for table in tables:
    query = "drop table {0}".format(table)
    try: session.execute(query)
    except Exception as e: print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()